In [322]:
import os 
import json 
import pandas as pd 
import polars as pl 
import numpy as np
from tqdm import tqdm 
import time 
from datetime import timedelta

import sys
import os




from upper_mpc_dro import step_upper_level, UMPCDataBuffer
from datetime import datetime, timezone


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [323]:
#time = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))
#upper_mpc_data_pan = pl.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))
#p1= pl.read_parquet("/home/alqua/data/data_vdfs/pump1_power_siso.par").group_by_dynamic(index_column="time", every="1h").agg(pl.col("pump1_power").mean())
#p4= pl.read_parquet("/home/alqua/data/data_vdfs/pump4_power_siso.par").group_by_dynamic(index_column="time", every="1h").agg(pl.col("pump4_power").mean())
#p3= pl.read_parquet("/home/alqua/data/data_vdfs/pump3_power_siso.par").group_by_dynamic(index_column="time", every="1h").agg(pl.col("pump3_power").mean())
#pumps_df = p1.join(p4, on="time").join(p3, on="time")
#baseline_df = (pumps_df.group_by("hours").agg(
#                    pl.col("energy_cons").mean().alias("mean_energy_cons"),
#                    pl.col("energy_cons").std().alias("std_energy_cons")
#                ).sort("hours"))

In [324]:
data_path_wsl = "/home/alqua/data/pump_station_data/"
data = pl.read_parquet('/home/alqua/papers/mbs_flexibility_mpc_rev_v1/mpc/input_data/aggregated_data_for_mpc.par')

In [325]:
start_date = pd.to_datetime("2023-02-10 00:00:00+00:00")
end_date = pd.to_datetime("2023-02-28 14:00:00+00:00")

data = data.filter((pl.col("time") >= start_date) & 
                   (pl.col("time") <= end_date)
                  ).with_columns(
                      pl.col("CO2Emission")/1000, #Kg/Kwh
                      pl.col("price")/1000 # 90 eur/ton -> 0.09eur per kg
                  )

data = data.with_columns(
    # Add Danish grid & tax surcharge: 0.166 EUR/kWh per element (business, 2024, excl. VAT)
    (pl.col("price") + 0.166), 
    (pl.col("CO2Emission") * 0.09))



In [326]:
new_cols_map = {
    'time': 'time_utc',
    'inflow': 'inflow_kf', 
    'inflow_0.1': 'inflow_q10', 
    'inflow_0.5': 'inflow_q50', 
    'inflow_0.9': 'inflow_q90', 
    'price': 'da_price',
    'CO2Emission': 'co2_progn'}

data = data.select(new_cols_map).rename(new_cols_map)


In [327]:
with open('../input_data/lower_mpc_coefficients.json', 'r') as json_file:
    models_coefficients = json.load(json_file)

In [328]:
umpc_data = UMPCDataBuffer()

In [329]:
umpc_ini = {
    'time_utc': np.nan,             # timestamp for 2023-02-10 01:00:00 UTC
    'qout': 360,                   # no data given
    'qin': 351.382888,                  # inflow_kf
    'qin_q10': 143.373214,              # inflow_q10
    'qin_q50': 330.127912,              # inflow_q50
    'qin_q90': 502.281903,              # inflow_q90
    'height_ref': 150,             # no data given
    'energy_ref': 20,             # no data given
    'co2_progn': 0.0882,                # co2_progn
    'da_price': 0.18384,                # da_price
    'objective': 0,              # no data given
    'opt_time_umpc':0          # no data given
}

In [330]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import warnings
import polars as pl 
import matplotlib.pyplot as plt 
import scienceplots
from matplotlib.ticker import AutoMinorLocator

umpc_data = UMPCDataBuffer()
umpc_data.initialize(umpc_ini)

warnings.filterwarnings('ignore', module='matplotlib')
plt.style.use('ieee')

slice_df = data.select(pl.col(["time_utc", "inflow_kf", "co2_progn","da_price", 'inflow_q10', 'inflow_q50','inflow_q90']))
inflow_kf = slice_df["time_utc", "inflow_kf"].upsample(every="1m", time_column= "time_utc").fill_null(strategy="forward")["inflow_kf"]

zs = 3 
N = 60
horizon = 24
step_size = 1
max_start = len(slice_df) - horizon + 1
start_index = 0
trigger = [1, 0, 0]

with tqdm(total=max_start, desc="Hour Steps") as pbar:
    start_index = 0
    while start_index < max_start:
        prices_values = slice_df["da_price"][start_index : start_index + horizon]
        co2_values = slice_df["co2_progn"][start_index : start_index + horizon]
        inflow_values = slice_df["inflow_kf"][start_index : start_index + horizon]
        
        start_time_umpc =  time.time()
        ############################################### Upper Level Optimization
        
        umpc_opt_results = step_upper_level(
            horizon=horizon,
            prices_values=prices_values,
            co2_progn_values=co2_values,
            inflow_values=inflow_values,
            h_init=umpc_data.data["height_ref"][-1],
            energy_init=umpc_data.data["energy_ref"][-1],
            Qout_init=umpc_data.data["qout"][-1])

        end_time_umpc = time.time() - start_time_umpc
        umpc_opt_results["opt_time_umpc"] = end_time_umpc
        umpc_opt_results["qin_q10"] = slice_df["inflow_q10"][start_index]
        umpc_opt_results["qin_q50"] = slice_df["inflow_q50"][start_index]
        umpc_opt_results["qin_q90"] = slice_df["inflow_q90"][start_index]
        umpc_opt_results["time_utc"] = slice_df["time_utc"][start_index]

        umpc_data.update(umpc_opt_results)

        ############################################### Lower Level Optimization
        start_index += 1
        if start_index == 24:
            break

        pbar.update(step_size)
    results_df = umpc_data.to_dataframe(save=False, file_path='../output_data/umpc_rain_shift_results.par', skip_ini=True)


Hour Steps:   0%|          | 1/423 [00:00<01:06,  6.35it/s]

-52.0


Hour Steps:   0%|          | 2/423 [00:00<01:05,  6.46it/s]

-52.0


Hour Steps:   1%|          | 3/423 [00:00<01:03,  6.57it/s]

...

Hour Steps:   5%|▍         | 21/423 [00:02<00:49,  8.18it/s]

-18.6
-11.4


Hour Steps:   5%|▌         | 23/423 [00:03<00:48,  8.18it/s]

4.4
6.1
12.7


Hour Steps:   5%|▌         | 23/423 [00:03<00:54,  7.38it/s]


In [332]:
results_df = umpc_data.to_dataframe(save=False, file_path='../output_data/umpc_rain_shift_results.par', skip_ini=True)

results_df

time_utc,qout,qin,qin_q10,qin_q50,qin_q90,height_ref,energy_ref,co2_progn,da_price,objective,opt_time_umpc
"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-02-10 03:00:00 UTC,54.7,323.3,14.392493,215.709795,377.961821,200.0,-0.0,0.1,0.1857,148452.1,0.148668
2023-02-10 04:00:00 UTC,29.2,331.0,31.300132,186.685908,352.843648,200.0,-0.0,0.1002,0.1915,134927.7,0.147707
2023-02-10 05:00:00 UTC,15.6,352.6,17.303789,190.353061,328.82951,200.0,-0.0,0.0932,0.2576,120499.3,0.13519
2023-02-10 06:00:00 UTC,35.8,380.6,-8.328892,171.130309,325.804499,185.6,5.9,0.1021,0.2938,120298.6,0.136267
2023-02-10 07:00:00 UTC,20.6,404.1,19.566682,157.491045,307.888566,70.0,0.3,0.0984,0.3088,117564.5,0.130353
…,…,…,…,…,…,…,…,…,…,…,…
2023-02-10 20:00:00 UTC,49.7,379.8,97.215689,269.602694,450.120956,120.3,1.8,0.0466,0.1961,100084.1,0.121929
2023-02-10 21:00:00 UTC,31.1,368.0,57.902199,226.973878,355.962801,117.7,1.0,0.0426,0.196,100041.1,0.120217
2023-02-10 22:00:00 UTC,16.6,359.4,157.00647,313.93065,478.41605,99.9,-0.0,0.0596,0.1985,99565.4,0.125328
